# How to use a VeinsGym Environment

In this tutorial, we will load the *Serpentine* example scenario using `VeinsGym` and run an agent within it.

## Dependencies

To run VeinsGym and the examples in this notebook, we need a few dependencies ready to run:

- [Sumo](https://www.eclipse.org/sumo/) (VeinsGym is tested with Sumo v1.6.0, other versions *may* work as well)
- [OMNeT++](https://omnetpp.org/) v5.6+ (Not yet tested with OMNeT++ v6)
- GNU `make` a `C++` compiler (ideally the one that you used to compile OMNeT++, check `opp_run -v`)
- [Google Protocol Buffers](https://developers.google.com/protocol-buffers) compiler (`protoc`)

If you have fulfilled these dependencies, running `protoc --version`, `sumo --version`, and `opp_run -v` should work fine.
Otherwise you may have to adjust your `$PATH` variable to include them.

## Installing VeinsGym

To run this notbook we first need to install the `veins_gym` package (e.g., to a virtual environment).
To build the *Serpentine* scenario in the next step, we will also need *Snamkemake*, so let's install that as well.

```bash
pip install veins_gym snakemake
```

## Checking out the *Serpentine* scenario

Next, we need a scenario to run.
A scenario contains the code and configuration to run a VANET experiment using Veins and the VeinsGym client code.
See https://github.com/tkn-tub/veins-gym/blob/master/doc/getting_started.md to find out how to write your own.

In this tutorial we are using the *Serpentine* example scenario.
You can clone it from https://github.com/tkn-tub/serpentine-env

```bash
git clone https://github.com/tkn-tub/serpentine-env
```

The *Serpentine* scenario brings all the veins, veins-vlc, and scenario-specific code with it.
Before you can use it, you have to compile it, though.
This is done with *Snakemake*.
We use all cores of the machine (`-jall`) to speed up the process.

```bash
cd serpentine-env
snakemake -jall
```

Now the scenario is ready and we can setup the the gyn environment to expose it to an agent.

## Register your scenario as a Gym Environment

We import the general `gym` module and the `veins_gym` connection module to register the scenario as a gym environment.
The most important part is to point the `scenario_dir` argument to the directory in which your `omnetpp.ini` files lives.

In [1]:
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_

In [2]:
import gym
import veins_gym

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
# Ensure OMNeT++ environment is available inside this kernel
import os, subprocess, shutil

def source_bash_env(script_path: str):
    """Source a bash script and import its exported environment into this process."""
    cmd = f"bash -lc 'source {script_path} && env -0'"
    out = subprocess.check_output(cmd, shell=True)
    for chunk in out.split(b"\x00"):
        if not chunk:
            continue
        k, _, v = chunk.partition(b"=")
        if k:
            os.environ[k.decode()] = v.decode()

# Source your OMNeT++ installation's setenv script
omnetpp_setenv = "/home/abhay/omnetpp-5.7.1/setenv"
if os.path.exists(omnetpp_setenv):
    source_bash_env(omnetpp_setenv)
    print("Sourced:", omnetpp_setenv)
else:
    raise FileNotFoundError(f"OMNeT++ setenv not found at {omnetpp_setenv}. Update the path if installed elsewhere.")

print("opp_run path:", shutil.which("opp_run"))
try:
    ver = subprocess.run(["opp_run", "-v"], check=False, capture_output=True, text=True).stdout.splitlines()[0]
    print(ver)
except Exception as e:
    print("Warning: opp_run not callable:", e)


Sourced: /home/abhay/omnetpp-5.7.1/setenv
opp_run path: /home/abhay/omnetpp-5.7.1/bin/opp_run
OMNeT++ Discrete Event Simulation  (C) 1992-2021 Andras Varga, OpenSim Ltd.


In [4]:
gym.register(
    id="veins-v1",
    entry_point="veins_gym:VeinsEnv",
    kwargs={
        "scenario_dir": "../scenario",
        "timeout": 3.0,
        "print_veins_stdout": True,
        # "user_interface": "Cmdenv",
        # "config": "General",
    },
)

Now we can create an `env` instance.
We can use the `gym.make` call or other supporting wrappers from various RL-toolkits.
Just give it the environment name we registered it with above.

In [5]:
env = gym.make("veins-v1")

We can use this Environment like any other one from OpenAI or such.
Veins will automatically be started on `env.reset()` and restarted once an experiment is through.

In [6]:
env.reset()
done = False
while not done:
    random_action = env.action_space.sample()
    observation, reward, done, info = env.step(random_action)

/home/abhay/anaconda3/envs/mp/lib/python3.13/site-packages/gym/utils/passive_env_checker.py:181: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed='NO SEED GIVEN'
  logger.warn(
/home/abhay/anaconda3/envs/mp/lib/python3.13/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
/home/abhay/anaconda3/envs/mp/lib/python3.13/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


From here on it is up to you.
You can extract observations, evaluate exisiting policies, or train agents online.
Have fun!

## Example: Mean Reward

Like before we want to choose random actions.
But this time, we are going to track the rewards and compute the mean in the end.
In production, your RL-toolkit will probably do this for you

In [7]:
env.reset()
done = False
rewards = []
while not done:
    random_action = env.action_space.sample()
    observation, reward, done, info = env.step(random_action)
    rewards.append(reward)
print("Number of steps taken:", len(rewards))
print("Mean reward:", sum(rewards) / len(rewards))

Number of steps taken: 3679
Mean reward: 0.6408397
